In [12]:
# load data and write out sentence and target
import pandas as pd

loaded_set = pd.read_excel("Dataset/"+"training.xlsx")
data_dict = {'sentence':loaded_set['Sentence'], 'target':loaded_set['MOS']}
data_set = pd.DataFrame.from_dict(data_dict)

data_set

'In der großen Kasernenanlage im Norden Kiels kam es zu Demonstrationen.'

In [10]:
import tensorflow_hub as hub
from transformers import AutoModel, AutoTokenizer
# german tokens for bert
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
#model = AutoModel.from_pretrained("dbmdz/bert-base-german-cased")

Downloading: 100%|██████████| 456/456 [00:00<00:00, 228kB/s]
Downloading: 100%|██████████| 240k/240k [00:00<00:00, 559kB/s]
Downloading: 100%|██████████| 59.0/59.0 [00:00<00:00, 59.0kB/s]


In [17]:
token = []
token_ids = []
for x in range(max(data_set.index)):
    token.append(tokenizer.tokenize(data_set['sentence'][1]))
    token_ids.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(data_set['sentence'][1])))



  'im',
  'Norden',
  'Kiel',
  '##s',
  'kam',
  'es',
  'zu',
  'Demonstrationen',
  '.'],
 ['In',
  'der',
  'großen',
  'Kas',
  '##ernen',
  '##anlage',
  'im',
  'Norden',
  'Kiel',
  '##s',
  'kam',
  'es',
  'zu',
  'Demonstrationen',
  '.'],
 ['In',
  'der',
  'großen',
  'Kas',
  '##ernen',
  '##anlage',
  'im',
  'Norden',
  'Kiel',
  '##s',
  'kam',
  'es',
  'zu',
  'Demonstrationen',
  '.'],
 ['In',
  'der',
  'großen',
  'Kas',
  '##ernen',
  '##anlage',
  'im',
  'Norden',
  'Kiel',
  '##s',
  'kam',
  'es',
  'zu',
  'Demonstrationen',
  '.'],
 ['In',
  'der',
  'großen',
  'Kas',
  '##ernen',
  '##anlage',
  'im',
  'Norden',
  'Kiel',
  '##s',
  'kam',
  'es',
  'zu',
  'Demonstrationen',
  '.'],
 ['In',
  'der',
  'großen',
  'Kas',
  '##ernen',
  '##anlage',
  'im',
  'Norden',
  'Kiel',
  '##s',
  'kam',
  'es',
  'zu',
  'Demonstrationen',
  '.'],
 ['In',
  'der',
  'großen',
  'Kas',
  '##ernen',
  '##anlage',
  'im',
  'Norden',
  'Kiel',
  '##s',
  'kam',
  'e

In [14]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(data_set['sentence'][1]))

[259,
 125,
 1793,
 7450,
 3868,
 3858,
 223,
 3832,
 8632,
 30886,
 1396,
 288,
 205,
 25880,
 566]